In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision.transforms import Resize, ToTensor, Normalize

from tqdm import tqdm

In [2]:
import sys
import gzip
import random
import platform
import warnings
import collections

import re
import requests
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import torchvision
import torch.optim as optim
from torchvision import transforms, utils

from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torchvision.transforms.functional import to_pil_image


In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import glob
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import glob
from tqdm import tqdm

In [4]:
def makeLabeling(pd,idx,name):
    if name == 'mask':
        if pd['gender'][idx] == 'male':
            if pd['age'][idx] < 30: return 0
            elif 30 <= pd['age'][idx] < 60: return 1
            else: return 2
        else:
            if pd['age'][idx] < 30: return 3
            elif 30 <= pd['age'][idx] < 60: return 4
            else: return 5
    elif name == 'incorrect':
        if pd['gender'][idx] == 'male':
            if pd['age'][idx] < 30: return 6
            elif 30 <= pd['age'][idx] < 60: return 7
            else: return 8
        else:
            if pd['age'][idx] < 30: return 9
            elif 30 <= pd['age'][idx] < 60: return 10
            else: return 11
    else:
        if pd['gender'][idx] == 'male':
            if pd['age'][idx] < 30: return 12
            elif 30 <= pd['age'][idx] < 60: return 13
            else: return 14
        else:
            if pd['age'][idx] < 30: return 15
            elif 30 <= pd['age'][idx] < 60: return 16
            else: return 17

In [5]:
original_csv = pd.read_csv('/opt/ml/input/data/train/train.csv')

# val train split 할 기준 세우기
import random
total = list(range(2700))
testidx = random.sample(total,int(2700*0.2))
testidx.sort()
print(len(testidx))
# 테스트용 2

tempidx = total
for i in testidx:
    tempidx.remove(i)

total = tempidx
validx = random.sample(total,int(2700*0.2))
validx.sort()
print(len(validx))
# valid용 2

trainidx = total
for i in validx:
    trainidx.remove(i)
print(len(trainidx))

540
540
1620


In [11]:
test_file_dict = {k:[] for k in ['id','path','file_name','absolute_path','label']}

IMAGE_PATH = '/opt/ml/input/data/train/images'

# val_img_label
for i in testidx:
    total_path = os.path.join(IMAGE_PATH,str(original_csv['path'][i]))
    for file_name in os.listdir(total_path):
        # 확장자 제거, 이름만 가져오기
        fn_split = file_name.split(".")
        if len(fn_split[0]) == 0: 
            continue
        name = re.findall("[a-z]+",fn_split[0])[0]
        
        # labeling
        label = makeLabeling(original_csv,i,name)
        test_file_dict['id'].append(original_csv['id'][i])
        test_file_dict['file_name'].append(file_name)
        test_file_dict['path'].append(original_csv['path'][i])
        test_file_dict['absolute_path'].append(os.path.join(total_path,file_name))
        test_file_dict['label'].append(label)

for k in test_file_dict:
    print(f"{k}: {len(test_file_dict[k])}")


id: 3780
path: 3780
file_name: 3780
absolute_path: 3780
label: 3780


In [12]:
image_label = pd.DataFrame(test_file_dict)
np.unique(image_label['label'])
image_label.to_csv('test_image_label.csv',index=False)

In [13]:
val_file_dict = {k:[] for k in ['id','path','file_name','absolute_path','label']}

IMAGE_PATH = '/opt/ml/input/data/train/images'

# val_img_label
for i in validx:
    total_path = os.path.join(IMAGE_PATH,str(original_csv['path'][i]))
    for file_name in os.listdir(total_path):
        # 확장자 제거, 이름만 가져오기
        fn_split = file_name.split(".")
        if len(fn_split[0]) == 0: 
            continue
        name = re.findall("[a-z]+",fn_split[0])[0]
        
        # labeling
        label = makeLabeling(original_csv,i,name)
        val_file_dict['id'].append(original_csv['id'][i])
        val_file_dict['file_name'].append(file_name)
        val_file_dict['path'].append(original_csv['path'][i])
        val_file_dict['absolute_path'].append(os.path.join(total_path,file_name))
        val_file_dict['label'].append(label)

for k in val_file_dict:
    print(f"{k}: {len(val_file_dict[k])}")


id: 3780
path: 3780
file_name: 3780
absolute_path: 3780
label: 3780


In [14]:
image_label = pd.DataFrame(val_file_dict)
np.unique(image_label['label'])
image_label.to_csv('val_image_label.csv',index=False)

In [15]:
train_file_dict = {k:[] for k in ['id','path','file_name','absolute_path','label']}

IMAGE_PATH = '/opt/ml/input/data/train/images'

# val_img_label
for i in trainidx:
    total_path = os.path.join(IMAGE_PATH,str(original_csv['path'][i]))
    for file_name in os.listdir(total_path):
        # 확장자 제거, 이름만 가져오기
        fn_split = file_name.split(".")
        if len(fn_split[0]) == 0: 
            continue
        name = re.findall("[a-z]+",fn_split[0])[0]
        
        # labeling
        label = makeLabeling(original_csv,i,name)
        train_file_dict['id'].append(original_csv['id'][i])
        train_file_dict['file_name'].append(file_name)
        train_file_dict['path'].append(original_csv['path'][i])
        train_file_dict['absolute_path'].append(os.path.join(total_path,file_name))
        train_file_dict['label'].append(label)

for k in train_file_dict:
    print(f"{k}: {len(train_file_dict[k])}")


id: 11340
path: 11340
file_name: 11340
absolute_path: 11340
label: 11340


In [16]:
image_label = pd.DataFrame(train_file_dict)
np.unique(image_label['label'])
image_label.to_csv('train_image_label.csv',index=False)